# First lab assignment


In [98]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, RepeatedKFold
from sklearn.metrics import silhouette_score, accuracy_score, classification_report, confusion_matrix, r2_score, root_mean_squared_error
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LinearRegression



## Task 1: Data description

In [99]:
# Load the earnings dataset
df = pd.read_csv('earnings.csv', sep=';')
# Print info about the dataset before and after dropping missing data
print('--- Before dropping missing data: ---')
print(df.info())
df = df.dropna()
print('--- After dropping missing data: ---')
print(df.shape)
df

--- Before dropping missing data: ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11000 entries, 0 to 10999
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 11000 non-null  int64  
 1   base               11000 non-null  float64
 2   bonus              11000 non-null  float64
 3   overtime_pay       11000 non-null  float64
 4   other              11000 non-null  float64
 5   sector             11000 non-null  int64  
 6   section_07         11000 non-null  int64  
 7   sex                11000 non-null  int64  
 8   education          11000 non-null  int64  
 9   contract           11000 non-null  int64  
 10  age                11000 non-null  int64  
 11  duration_total     11000 non-null  float64
 12  duration_entity    11000 non-null  float64
 13  duration_nominal   11000 non-null  float64
 14  duration_overtime  11000 non-null  float64
dtypes: float64(8), int64(7)
memory u

,id,base,bonus,overtime_pay,other,sector,section_07,sex,education,contract,age,duration_total,duration_entity,duration_nominal,duration_overtime
0,192064,26651.53,0.00,0.00,0.00,1,3,2,4,1,49,33.03,7.06,1524.15,0.0
1,25495,40168.50,1500.00,0.00,3414.32,1,2,1,1,1,36,10.07,6.01,1562.40,0.0
2,142164,20134.80,0.00,0.00,1700.41,1,2,2,4,1,52,28.08,19.05,1816.00,0.0
3,198034,16475.00,0.00,0.00,1305.00,1,2,2,5,1,55,35.07,11.01,1816.00,0.0
4,144990,34797.60,0.00,1893.35,3118.73,1,2,2,2,1,50,27.00,19.01,722.80,63.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10995,41597,36573.22,1323.65,1276.80,3337.32,1,2,2,2,1,50,27.02,27.02,766.80,40.0
10996,120022,28280.00,5470.00,0.00,2385.13,1,1,2,4,1,41,18.08,15.09,1792.00,0.0
10997,41800,109316.96,0.00,0.00,9042.58,1,1,2,2,1,47,20.02,1.07,1656.00,0.0
10998,153849,57721.35,6950.00,0.00,4906.32,1,1,2,2,1,47,28.01,21.10,1784.00,0.0


We can see that the dataset has 11000 observations and 15 columns, with the same variables that were discussed in the assignment text. There does not seem to be any missing data, because we have the same number of observations after calling `dropna()` on our dataframe. From looking at the dataset and the description provided in the assignment we can see that we have 6 qualitative variables, which are `id, sector, section_07, sex, education, contract`, while the other 9 variables are quantitive, and they are `base, bonus, overtime_pay, other, age, duration_total, duration_entity, duration_nominal, duration_overtime`. We can see that the quantitive variables mostly contain information about the different types of pay and durations of time the employee has worked. Now, for each variable (excluding the `id`) we will use frequency tables (if it's qualitative) or descriptive statistics (if it's quantitative).

In [100]:
# For qualitative variables
qualitative_vars = ['sector', 'section_07', 'sex', 'education', 'contract']
for var in qualitative_vars:
    print(f"--- Frequency table for {var}: ---")
    print(df[var].value_counts())
    print()
# For quantitative variables
quantitative_vars = df.columns.difference(qualitative_vars + ['id'])
for var in quantitative_vars:
    print(f"--- Descriptive statistics for {var}: ---")
    print(df[var].describe())
    print()

--- Frequency table for sector: ---
sector
1    10548
2      452
Name: count, dtype: int64

--- Frequency table for section_07: ---
section_07
2    5867
3    2732
1    2401
Name: count, dtype: int64

--- Frequency table for sex: ---
sex
2    8289
1    2711
Name: count, dtype: int64

--- Frequency table for education: ---
education
2    6633
4    1983
5     906
3     680
1     430
6     368
Name: count, dtype: int64

--- Frequency table for contract: ---
contract
1    9306
2    1694
Name: count, dtype: int64

--- Descriptive statistics for age: ---
count    11000.000000
mean        42.470182
std         10.012140
min         19.000000
25%         34.000000
50%         43.000000
75%         51.000000
max         77.000000
Name: age, dtype: float64

--- Descriptive statistics for base: ---
count     11000.000000
mean      33376.738065
std       19276.551638
min          10.000000
25%       20995.115000
50%       31341.245000
75%       41348.290000
max      241624.390000
Name: base, dtype:

We can learn some things about our dataset from these. First of all, we see the vast majority of the data is gathered from the public sector, with most of the observations being in Education. The majority of the observations also come from women. We also see that a lot of observations have a higher education (around 65%), which is larger than on average. For the quantitive variables we can see the means and variances, but we will learn more about them by looking at their plots.

In [101]:
num_vars = len(quantitative_vars)
fig = make_subplots(rows=num_vars, cols=1, subplot_titles=quantitative_vars, vertical_spacing=0.02)

distributions_to_fit = {
    'age': stats.norm,
    'base': stats.norm,
    'bonus': stats.expon,
    'duration_overtime': stats.expon,
    'overtime_pay': stats.expon,
}

for i, var in enumerate(quantitative_vars):
    data = df[var]

    # Add a normalized histogram
    fig.add_trace(go.Histogram(x=data, histnorm='probability density', name=var, showlegend=False), row=i + 1, col=1)

    # Fit a distribution from the dictionary
    if var in distributions_to_fit:
        dist = distributions_to_fit[var]
    else:
        continue
    args = dist.fit(data.to_numpy())
    print(f"Fitted parameters for {var}: {args}")
    # Plot the fitted distribution
    x = np.linspace(data.min(), data.max(), 1000)
    p = dist.pdf(x, *args)
    fig.add_trace(go.Scatter(x=x, y=p, mode='lines', name='Fitted distribution'), row=i + 1, col=1)

fig.update_layout(
    title='Distributions of quantitative variables',
    showlegend=True,
    width=1400,
    height=400 * len(quantitative_vars),
)
fig.show()

Fitted parameters for age: (np.float64(42.47018181818182), np.float64(10.011685080382838))
Fitted parameters for base: (np.float64(33376.738064545454), np.float64(19275.675411081316))
Fitted parameters for bonus: (0.0, 2128.4861763636363)
Fitted parameters for duration_overtime: (0.0, 47.70232454545454)
Fitted parameters for overtime_pay: (0.0, 1679.2739227272725)


The plots display the distributions of the quantitative variables in the dataset, each shown as a normalized histogram, some overlaid with a fitted probability distribution curve. For variables such as `age` and `base`, a normal distribution was fitted, while for variables like `bonus`, `overtime_pay`, and `duration_overtime`, an exponential distribution was used.

From these plots, we can observe the following:
* `age`: the normal distribution is not a great fit, because of the fluctuating patterns in age distribution, which we can observe f.ex. in population pyramids.
* `base`: the distribution is right-skewed, with most employees earning around the mean, but a long tail of higher salaries. The normal fit captures the central tendency but does not account for the long tail on the right.
* `bonus`, `overtime_pay`, and `duration overtime`: these variables have a large number of observations near zero and a long tail of higher values. The exponential fit is appropriate here because most employees receive little or no bonus or overtime, while a few receive much higher amounts. We can see that the fit is not great for `duration_overtime`.
* `other`: the other forms of renumeration follow a kind of normal distribution with, once again, a long tail, but there is also a high number of employees earning zero in this category.
* `duration_total`: seems to be highly correlated with the distribution of ages, which makes sense.

## Task 2: Clustering

For clustering we will try to use variables which can be perceived as more 'fundamental' than others, and we exclude:
* `id`: obviously does not provide any 'real' data about an observation
* `bonus`: these can be very variable and depend on a lot of factors
* `overtime_pay`: can be very variable and is already correlated with the base pay in some way
* `duration_entity`: excluded, because `duration_total` seems more fundamental for a worker than the time spent with the company
* `duration_nominal` and `duration_overtime`: the first correlates with the pay, and the other is excluded because we also exclude overtime pay
* `other`: a very broad and variable category of pay which might not help with clustering

For the categorical variables we will use one-hot encoding (three of them only have values of 1 and 2, so this will essentially convert them to a binary column). An exception to this will be the education variable - even though it is categorical, we will leave it as is, because there is an order to the categories, with lower numbers describing a higher education.

To avoid the quantitive variables overtaking the clustering because of their magnitude (0-1 for categorical one-hot encoded variables vs around 30k for the base salary) we will use min-max scaling so that all the variables lie in the 0-1 range.

We will use the k-means algorithm using the Euclidean metric to find the clusters.

In [102]:
# To perform clustering, we will first need to encode the qualitative variables using one-hot encoding
one_hot_vars = ['sector', 'section_07', 'sex', 'contract']
df_encoded = pd.get_dummies(df, columns=one_hot_vars, drop_first=True, dtype=int)
# Now we scale quantitive variables using MinMaxScaler
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df_encoded), columns=df_encoded.columns)
df_scaled = df_scaled.drop(columns=['id', 'other', 'bonus', 'overtime_pay', 'duration_entity', 'duration_nominal', 'duration_overtime'])  # Drop the not needed columns
print(df_scaled.info())

# Now we can perform clustering using KMeans
# We determine the best number of clusters using the silhouette score
best_n_clusters = 0
best_silhouette_score = -1
for n_clusters in range(2, 10):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(df_scaled)
    silhouette_avg = silhouette_score(df_scaled, cluster_labels)
    if silhouette_avg > best_silhouette_score:
        best_n_clusters = n_clusters
        best_silhouette_score = silhouette_avg
    print(f"For n_clusters = {n_clusters}, the average silhouette score is: {silhouette_avg:.4f}")

print(f"The best number of clusters is: {best_n_clusters} with an average silhouette score of: {best_silhouette_score:.4f}")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11000 entries, 0 to 10999
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   base            11000 non-null  float64
 1   education       11000 non-null  float64
 2   age             11000 non-null  float64
 3   duration_total  11000 non-null  float64
 4   sector_2        11000 non-null  float64
 5   section_07_2    11000 non-null  float64
 6   section_07_3    11000 non-null  float64
 7   sex_2           11000 non-null  float64
 8   contract_2      11000 non-null  float64
dtypes: float64(9)
memory usage: 773.6 KB
None
For n_clusters = 2, the average silhouette score is: 0.3647
For n_clusters = 3, the average silhouette score is: 0.3726
For n_clusters = 4, the average silhouette score is: 0.4035
For n_clusters = 5, the average silhouette score is: 0.4466
For n_clusters = 6, the average silhouette score is: 0.4627
For n_clusters = 7, the average silhouette score i

## Task 3: Classification

For the classification task we will use boosted decision trees. As before, we use one-hot encoding for the categorical variables and then scale the rest of the variables to the 0-1 range using min-max scaling. To see how our model will perform on unseen data, we use `train_test_split()` and use 20% of our dataset as a test set. To find the best model for our problem we will use the grid search implemented in scikit-learn. We will use stratified cross validation and the scoring metric will be the weighted F1-Score of the model. We don't use the accuracy because the classes are skewed (there are a more people with a higher education in the dataset). To see which features are most important for the classification we will use permutation feature importance.

In [103]:
# First, we use one-hot encoding for the qualitative variables (excluding 'education' because we will drop it later)
qualitative_vars_without_education = qualitative_vars.copy()
qualitative_vars_without_education.remove('education')
df_encoded = pd.get_dummies(df, columns=qualitative_vars_without_education, drop_first=True, dtype=int)
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df_encoded), columns=df_encoded.columns)
df_scaled = df_scaled.drop(columns=['id', 'education'])  # Drop the 'id' and 'education' columns

# X will be the features and y will be the target variable (education <= 2)
X = df_scaled.to_numpy()
y = df['education'].to_numpy()
# y should be 1 when education <= 2, and 0 otherwise
y = np.where(y <= 2, 1, 0)

# Split the data into training and testing sets to assess the classifier on unseen data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the parameter grid for AdaBoost
param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.3, 0.5, 0.7, 1.0],
}
# Create an AdaBoost classifier
ada = AdaBoostClassifier()

# Perform grid search with cross-validation
grid_search_ada = GridSearchCV(ada, param_grid, cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42), scoring='f1_weighted', n_jobs=-1)
grid_search_ada.fit(X_train, y_train)
model = grid_search_ada.best_estimator_
print(f"Best parameters for AdaBoost: {grid_search_ada.best_params_}")

# Evaluate the model on the test set
y_pred_ada = model.predict(X_test)
accuracy_ada = accuracy_score(y_test, y_pred_ada)
print(f"AdaBoost Accuracy: {accuracy_ada:.4f}")
print(classification_report(y_test, y_pred_ada))
print(confusion_matrix(y_test, y_pred_ada))

# Calculate permutation importance
perm_importance = permutation_importance(model, X_test, y_test, n_repeats=15, random_state=42)
# Sort the importances in descending order
sorted_idx = perm_importance.importances_mean.argsort()[::-1]
# Create a bar plot of the permutation importances
fig = px.bar(x=perm_importance.importances_mean[sorted_idx], y=df_scaled.columns[sorted_idx], orientation='h', title='Permutation Importances')
fig.update_layout(xaxis_title='Importance', yaxis_title='Features')
fig.show()


Best parameters for AdaBoost: {'learning_rate': 1.0, 'n_estimators': 200}
AdaBoost Accuracy: 0.8691
              precision    recall  f1-score   support

           0       0.81      0.81      0.81       747
           1       0.90      0.90      0.90      1453

    accuracy                           0.87      2200
   macro avg       0.85      0.85      0.85      2200
weighted avg       0.87      0.87      0.87      2200

[[ 602  145]
 [ 143 1310]]


The model with the best parameters achieved an accuracy of 86.91%, but performed better on the majority class 1 (which are the observations with higher education) with higher precision and recall (90% each) compared to class 0 (80% each). The confusion matrix reveals that the model makes more false positive errors (predicting class 1 when it's 0) than false negative errors, which is likely influenced by the class imbalance, because as we noted at the start, the majority of observations has a higher education in the dataset. When looking at the feature importances, some make sense, like the importance of the base pay (we expect that educated people get paid more on average), while the feature describing how many hours a worker has actually worked does not immediately seem like one that would be important for predicting their education. A few of the features have very low importances, like the amount of overtime or the sex of the worker.

## Task 4: Regression

We will try to predict the base salary using a linear regression model. We will use all other variables, the categorical variables (with the exception of education again) will be one-hot encoded, and the others will be scaled using StandardScaler. Of course, we will not be using the `id` column.

First, we will create a model using the whole dataset with statsmodels, which gives us access to p-values of the coefficients for each variable. Then, we will remove the variable with the highest p-value if it is above a significance level $\alpha$, then refit the model without it. We continue to do this until we only have features which are significant.

Then, we fit the model with these significant features and test it using repeated k-fold cross validation.


In [104]:
one_hot_vars = ['sector', 'section_07', 'sex', 'contract']
df_encoded = pd.get_dummies(df, columns=one_hot_vars, drop_first=True, dtype=int)
# Add a constant term for the intercept
df_encoded['const'] = 1
# Now we scale quantitive variables using StandardScaler
scaler = StandardScaler()
df_encoded[quantitative_vars] = scaler.fit_transform(df_encoded[quantitative_vars])
df_scaled = df_encoded.drop(columns=['id'])  # Drop the 'id' column

# First, we try to create a linear regression model using the whole dataset to remove insignificant variables
X = df_scaled.drop(columns=['base'])
y = df['base']
results = sm.OLS(y, X).fit()

# Print the detailed summary of the original model
print(results.summary())
print("\n")
pvalues = results.pvalues

# Set the significance level to 0.05
alpha = 0.05 

# Sort the p-values in ascending order
sorted_pvalues = pvalues.sort_values()
# Remove the feature with the highest p-value (the least significant feature)
while sorted_pvalues.iloc[-1] > alpha:
    feature_to_remove = sorted_pvalues.index[-1]
    X = X.drop(columns=[feature_to_remove])
    # Refit the model
    results = sm.OLS(y, X).fit()
    # Print the summary of the updated model
    print(f"--- Updated model after removing {feature_to_remove} ---")
    print(results.summary())
    print("\n")
    pvalues = results.pvalues
    sorted_pvalues = pvalues.sort_values()

X = X.to_numpy()
y = y.to_numpy()

# Create a linear regression model and validate it using RepeatedKFold
model = LinearRegression(fit_intercept=False)
kf = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)
rmse_scores = []
r2_scores = []

# Loop through the folds
for train_index, test_index in kf.split(X):
    # Split the data into training and testing sets
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    # Calculate the R2 score and RMSE for the current fold
    r2 = r2_score(y_test, y_pred)
    rmse = root_mean_squared_error(y_test, y_pred)
    r2_scores.append(r2)
    rmse_scores.append(rmse)

# Calculate the mean and standard deviation of the scores
mean_rmse = np.mean(rmse_scores)
std_rmse = np.std(rmse_scores)
mean_r2 = np.mean(r2_scores)
std_r2 = np.std(r2_scores)
print(f"Mean RMSE: {mean_rmse:.4f} ± {std_rmse:.4f}")
print(f"Mean R2: {mean_r2:.4f} ± {std_r2:.4f}")

                            OLS Regression Results                            
Dep. Variable:                   base   R-squared:                       0.494
Model:                            OLS   Adj. R-squared:                  0.494
Method:                 Least Squares   F-statistic:                     767.1
Date:                Mon, 05 May 2025   Prob (F-statistic):               0.00
Time:                        16:51:44   Log-Likelihood:            -1.2039e+05
No. Observations:               11000   AIC:                         2.408e+05
Df Residuals:                   10985   BIC:                         2.409e+05
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
bonus              1661.1337    136.82

The mean RMSE is about 13737, while the mean $R^2$ is around 0,49. Considering the base pay has a mean value of around 33377 and a standard deviation of 19277, this typical prediction error is quite substantial, and while the model has some predictive capability (capturing nearly half the variance of the data), the predictions still have a large average error relative to the overall distribution and spread of the values. The relatively small standard deviations on the cross-validation scores (±886 for RMSE, ±0.04 for $R^2$) suggest this level of performance is fairly consistent across different subsets of the data tested using cross validation. This shows that this simple linear model does not capture the complexities of our dataset.

Examining the model's coefficients allows us to understand the relationships it has identified between various features and the base salary:
* larger bonuses, overtime pay and other renumerations all show positive coefficients, which means they contribute positively to a higher base salary, which makes sense intuitively
* an older employee, or one which worked a longer time tends to get paid more, which also agrees with our intuition that more senior employees typically have higher salaries
* education has a negative coefficient, which in our case means that less educated workers have lower salaries (since in our data lower education feature means having a higher level of education), which seems generally expected
* the negative coefficients for the `section` features suggest that workers in Education and Human Health and Social Work Activities tend to earn less than those in Public Administration and Defence; Compulsory Social Security, in particular the coefficient for Education is a lot more negative, implying significantly lower compensation for workers in that field
* working in a contract for a definite period also seems to be correlated with lower salaries, which we could also link to the fact that senior staff, who often earn more, are more likely to hold indefinite-period contracts
* the coefficient for `sex_2` is negative, which tells us that the model predicts lower base salaries for women compared to men